In [1]:
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, Column, Integer, String, Float
Base = declarative_base()
import os

In [2]:
class Measurement(Base):
    __tablename__='measurement'
    measurement_id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

In [3]:
class Station(Base):
    __tablename__='station'
    station_id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [19]:
# Read csv
measurements_df = pd.read_csv('Resources\\clean_hawaii_measurements.csv')
measurements_df.rename(columns={'Unnamed: 0':'measurement_id'},inplace=True)
measurements_df.set_index(keys='measurement_id',inplace=True)
stations_df = pd.read_csv('Resources\\clean_hawaii_stations.csv')
stations_df.rename(columns={'Unnamed: 0':'station_id'},inplace=True)
stations_df.set_index(keys='station_id',inplace=True)

In [20]:
stations_df.head()

,station,name,latitude,longitude,elevation
station_id,,,,,
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [21]:
# Create/Update db and tables
engine = create_engine('sqlite:///hawaii.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)
conn = engine.connect()
measurements_df.to_sql(name='measurement',con=conn,flavor='sqlite',if_exists='replace')
stations_df.to_sql(name='station',con=conn,flavor='sqlite',if_exists='replace')

#conn.close()


C:\Users\Michael\Anaconda3\lib\site-packages\pandas\io\sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)
